In [2]:
import numpy as np

data = np.load("mit6.8610-project/data/toy_data/mutseqs.npy")
embeddings = np.load("mit6.8610-project/data/toy_data/ESMEmbed.npy")


In [3]:
embeddings = np.squeeze(embeddings,axis=1)

In [4]:
print(embeddings.shape)

(188, 163, 1280)


In [40]:
# print(data[:5],data.shape)
# print(embeddings[:5],embeddings.shape)
ngenes = 10
glen = embeddings.shape[1]//ngenes
npatients = embeddings.shape[0]
nembedSz = embeddings.shape[2]

cutpoints = [glen*i for i in range(1,ngenes)]


cuts = np.split(embeddings,cutpoints,axis=1)

for i,cut in enumerate(cutpoints):
    if(i+1 < len(cutpoints)):
        print(cutpoints[i+1]-cutpoints[i],end = ',')

print(embeddings.shape[1] - cutpoints[i])
def average_embeddings(slice_):
    return np.mean(slice_,axis=1)

print(cutpoints)

# 

# cuts = np.array(list(map(average_embeddings,cuts)))
# genes = np.stack(cuts)
# print(genes.shape)
# print(np.array(cuts).shape)
# x = [[np.mean(cut[0],axis=0),np.mean(cut[1],axis=0)] for cut in cuts]
# print(cuts.shape)
# np.save("mit6.8610-project/data/toy_data/EmbeddingsDivbyGenes_6.npy",cuts)
# x = [np.concatenate((cut[0],cut[1])) for cut in x]
# x = np.array(x)


16,16,16,16,16,16,16,16,19
[16, 32, 48, 64, 80, 96, 112, 128, 144]


# Graph Attention

In [26]:
# !pip install networkx
import networkx as nx

graph = nx.erdos_renyi_graph(ngenes, 0.5, seed=2, directed=False)

In [29]:
print(graph.nodes,graph.edges)
print(cuts[0].shape)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] [(0, 3), (0, 4), (0, 8), (1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 4), (3, 8), (3, 9), (4, 5), (4, 6), (5, 6), (7, 8)]
(188, 16, 1280)


In [30]:
import torch
# import dgl
from torch import cuda

if cuda.is_available():
    device = torch.device('cuda')
else:
    print("this is running in cpu!")
    device = torch.device('cpu')
from torch_geometric.utils.convert import from_networkx

# data = cuts.transpose(1,0,2)
# print(data.shape)





In [42]:
# def build_graph(node_embedding):
#     G = from_networkx(graph)
    
#     G.x = torch.Tensor(node_embedding)

#     G = G.to(device)
#     return G
# graphs = [build_graph(embedding) for embedding in data]
# print(len(graphs))

G = from_networkx(graph)
graphs = [G]*npatients

In [54]:
hparams = {
    "input_size":nembedSz,
    "hsz":500,
    "alpha": 0.2,
    "num_heads":5,
    "bsz":16,
    "num_epochs":100,
    "lr":1e-3,
    "ngenes":10,
    "geneLengths": [16,16,16,16,16,16,16,16,16,19]
}

In [59]:
# phenotype_table = np.load("seq_table.npy",allow_pickle=True)
# Y = phenotype_table[:,2].astype(np.int64)
# print(Y)
# Y = torch.nn.functional.one_hot(torch.Tensor(Y),num_classes=2)

Y = np.random.binomial(1,0.5,size=(npatients))
print(len(cuts),cuts[0].shape)

dataset = list(zip(cuts,Y))

# from dgl.data.utils import split_dataset
from sklearn.model_selection import train_test_split
train_dataset, valid_dataset = train_test_split(dataset, train_size=0.8)
valid_dataset, test_dataset = train_test_split(valid_dataset, train_size=0.5)

# from torch.utils.data import DataLoader
# def collate(samples):
#         graphs, labels = map(list, zip(*samples))
#         batched_graph = dgl.batch(graphs)
#         return batched_graph, torch.tensor(labels)
    
# train_loader = DataLoader(train_dataset, batch_size=hparams["bsz"], shuffle=True)
# valid_loader = DataLoader(valid_dataset, hparams["bsz"], shuffle=True,
#                      collate_fn=collate)
# test_loader = DataLoader(test_dataset, hparams["bsz"], shuffle=True,
#                      collate_fn=collate)


10 (188, 16, 1280)


In [55]:
from torch_geometric.nn import GCNConv, GATv2Conv
from torch import nn

class GeneEmbedding(nn.Module):
    def __init__(self,gene_length):
        super(GeneEmbedding, self).__init__()
        self.weighted_average_layer = nn.Linear(gene_length,1)
    
    def forward(self,x):
        return self.weighted_average_layer(x)

class GAT_Protein(nn.Module):
    def __init__(self,hparams):
        super(GAT_Protein, self).__init__()
        self.weightedAverage = []
        for i in range(hparams['ngenes']):
            self.weightedAverage.append(GeneEmbedding(hparams['geneLengths'][i]))
        self.averagedGenes = [0]*hparams['ngenes']
        
        self.input_layer = nn.Linear(hparams["input_size"],hparams["hsz"])
        self.GATLayer1 = GATv2Conv(hparams["hsz"],hparams["hsz"])
        self.output_layer1 = nn.Linear(hparams["hsz"]*hparams["ngenes"],hparams["hsz"])
        self.output_layer = nn.Linear(hparams["hsz"],1)
        self.leakyrelu = nn.LeakyReLU(hparams["alpha"])
        # self.output_layer = nn.functional.softmax(hsz,1)
        
    def forward(self, gene_list, edge_index):
        
        # print(x)
        
        for i,gene in enumerate(gene_list):
            self.averagedGenes[i] = self.weightedAverage[i](gene)
            
        
        x = torch.concatenate(self.averagedGenes,axis=0)
        print(x.shape)
        
        x = torch.relu(self.input_layer(x))
        
        x = self.leakyrelu(self.GATLayer1(x, edge_index))
        # print("Before Flatten",x,x.shape)
        x = torch.flatten(x)
        # print("After Flatten",x.shape)
        x = torch.relu(self.output_layer1(x))
        x = self.output_layer(x)
        # print("Before Softmax",x,x.shape)
        # x = nn.functional.softmax(x)
        # x = nn.functional.sigmoid(x)
        # print("After Softmax",x)
        return x
    
    # def avg_attention_heads(self, x):
        # return torch.mean(x, dim = 1)
        
    

In [56]:
# from torchsummary import summary
print(hparams)


model = GAT_Protein(hparams).to(device)
# summary(model)


{'input_size': 1280, 'hsz': 500, 'alpha': 0.2, 'num_heads': 5, 'bsz': 16, 'num_epochs': 100, 'lr': 0.001, 'ngenes': 10, 'geneLengths': [16, 16, 16, 16, 16, 16, 16, 16, 16, 19]}


In [58]:
print(len(cuts),cuts[0].shape)

10 (188, 16, 1280)


In [34]:
from tqdm import tqdm
def train(model, train_loader):
    model.train()
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=hparams["lr"])

    for epoch in range(hparams["num_epochs"]):
        train_loss = []
        
        for iter, (bg, label) in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            outputs = model(bg.x, bg.edge_index).double()
            # print(label)
            # print(outputs)
            label = torch.tensor([label]).to(device).double()
            
            # print(outputs.shape,label.shape)
            # print(outputs,label)
            # if iter == 2:
            #     break
            loss = loss_fn(outputs,label)
            # print("Loss",loss)
            loss.backward()
            
            optimizer.step()
            
        
            train_loss.append(loss.item())
        # for param in model.parameters():
        #     print(param.grad)
        
#         val_loss = []
#         model = model.eval()
#         with torch.no_grad():
#             for iter, (bg, label) in enumerate(tqdm(val_loader)):
#                 outputs = model(bg).double()
#                 label = torch.tensor(label).double()
#                 loss = loss_fn(outputs,label)
#                 val_loss.append(loss.item())
        
        train_loss = np.mean(train_loss)
        # val_loss = np.mean(val_loss)
    
        # print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')
        print(f'Epoch {epoch}: train loss {train_loss}')

train(model,train_dataset)


100%|████████████████████████████████████████████| 8/8 [00:00<00:00, 196.58it/s]


Epoch 0: train loss 2.4230113250869336


100%|████████████████████████████████████████████| 8/8 [00:00<00:00, 183.49it/s]


Epoch 1: train loss 0.7376279922127005


100%|████████████████████████████████████████████| 8/8 [00:00<00:00, 165.92it/s]


...

Epoch 97: train loss 0.0016013607930138781


100%|████████████████████████████████████████████| 8/8 [00:00<00:00, 178.94it/s]


Epoch 98: train loss 0.00151324933675166


100%|████████████████████████████████████████████| 8/8 [00:00<00:00, 194.27it/s]

Epoch 99: train loss 0.0014265746024402935
